
# IFCD104 – Tema 2: Autoencoder con Keras (MNIST)

Este cuaderno implementa un **autoencoder** sencillo para **reconstruir imágenes** del dataset **MNIST**.  
Incluye:
- Carga y **preprocesado** de datos.
- Construcción de un **autoencoder** `Encoder → Latent → Decoder`.
- **Entrenamiento** y **curvas de pérdida**.
- Visualización **original vs reconstruido**.
- Variante **Denoising Autoencoder** (opcional) añadiendo ruido a las entradas.
- Guardado y carga del modelo en formato `.keras` (ruta **local**).


## 1) Importaciones

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Utilidades
def show_images_grid(images, titles=None, nrows=2, ncols=8, suptitle=None):
    plt.figure(figsize=(ncols*1.2, nrows*1.2))
    idx = np.random.choice(len(images), size=nrows*ncols, replace=False)
    for i, j in enumerate(idx, start=1):
        plt.subplot(nrows, ncols, i)
        plt.imshow(images[j].squeeze(), cmap='gray')
        if titles is not None:
            plt.title(str(titles[j]))
        plt.axis('off')
    if suptitle:
        plt.suptitle(suptitle)
    plt.show()


## 2) Carga y preprocesado de MNIST

In [ ]:

(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

# Normalizar a [0,1] y añadir canal (1)
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, -1)  # (N, 28, 28, 1)
x_test  = np.expand_dims(x_test, -1)

print("Shapes -> train:", x_train.shape, " test:", x_test.shape)

# Muestra rápida
show_images_grid(x_train, suptitle="Muestras MNIST (train)")



## 3) Definición del Autoencoder
Arquitectura compacta y didáctica:
- **Encoder**: Conv→Pool→Conv→Pool→Flatten→Dense(latent)
- **Latent**: dimensión 32
- **Decoder**: Dense→Reshape→ConvTranspose→UpSampling→Conv(sigmoid)

> *Nota*: Para reconstrucción, usamos `loss='binary_crossentropy'` con imágenes normalizadas en `[0,1]`.


In [ ]:

latent_dim = 32

# Encoder
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
latent = layers.Dense(latent_dim, name='latent')(x)

# Decoder
x = layers.Dense(7*7*64, activation='relu')(latent)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D()(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D()(x)
outputs = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(inputs, outputs, name="mnist_autoencoder")
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()


## 4) Entrenamiento

In [ ]:

history = autoencoder.fit(
    x_train, x_train,            # objetivo: reconstruir la propia entrada
    validation_split=0.1,
    epochs=5,                    # aumenta si quieres mejor reconstrucción
    batch_size=128,
    verbose=1
)


## 5) Curvas de pérdida

In [ ]:

plt.figure()
plt.plot(history.history['loss'], label='loss_train')
plt.plot(history.history['val_loss'], label='loss_val')
plt.xlabel("Época")
plt.ylabel("Pérdida (binary_crossentropy)")
plt.title("Evolución de la pérdida (Autoencoder)")
plt.legend()
plt.show()


## 6) Reconstrucciones en test

In [ ]:

# Reconstrucciones
x_test_pred = autoencoder.predict(x_test[:2000], verbose=0)

# Mostrar originales y reconstrucciones
n = 8
plt.figure(figsize=(n*1.4, 3))
for i in range(n):
    # Original
    plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].squeeze(), cmap='gray')
    plt.axis('off')
    if i == 0: plt.ylabel("Original")
    # Reconstruido
    plt.subplot(2, n, n+i+1)
    plt.imshow(x_test_pred[i].squeeze(), cmap='gray')
    plt.axis('off')
    if i == 0: plt.ylabel("Reconstruido")
plt.suptitle("Original vs Reconstruido (Autoencoder)")
plt.show()

# Error de reconstrucción medio (MSE) en 2k ejemplos
mse = np.mean((x_test_pred - x_test[:2000])**2)
print("MSE medio de reconstrucción (2k test):", float(mse))



## 7) Denoising Autoencoder (opcional)
Agregamos **ruido gaussiano** a la entrada y entrenamos para reconstruir la **imagen limpia**.


In [ ]:

noise_std = 0.4
x_train_noisy = np.clip(x_train + noise_std*np.random.normal(size=x_train.shape), 0., 1.)
x_test_noisy  = np.clip(x_test  + noise_std*np.random.normal(size=x_test.shape),  0., 1.)

# Muestra con ruido
show_images_grid(x_train_noisy, suptitle="MNIST con ruido (train)")

# Entrenamos un modelo idéntico desde cero para denoising
inputs_n = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs_n)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
latent_n = layers.Dense(latent_dim, name='latent_n')(x)

x = layers.Dense(7*7*64, activation='relu')(latent_n)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D()(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D()(x)
outputs_n = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)

denoiser = keras.Model(inputs_n, outputs_n, name="mnist_denoiser")
denoiser.compile(optimizer='adam', loss='binary_crossentropy')

history_d = denoiser.fit(
    x_train_noisy, x_train,
    validation_split=0.1,
    epochs=3,
    batch_size=128,
    verbose=1
)


## 8) Resultados del Denoiser

In [ ]:

plt.figure()
plt.plot(history_d.history['loss'], label='loss_train')
plt.plot(history_d.history['val_loss'], label='loss_val')
plt.xlabel("Época")
plt.ylabel("Pérdida (binary_crossentropy)")
plt.title("Evolución de la pérdida (Denoiser)")
plt.legend()
plt.show()

# Reconstrucciones desde imágenes ruidosas
x_test_denoised = denoiser.predict(x_test_noisy[:2000], verbose=0)

n = 8
plt.figure(figsize=(n*1.4, 4.5))
for i in range(n):
    # Ruidosa
    plt.subplot(3, n, i+1)
    plt.imshow(x_test_noisy[i].squeeze(), cmap='gray')
    plt.axis('off')
    if i == 0: plt.ylabel("Ruidosa")
    # Denoised
    plt.subplot(3, n, n+i+1)
    plt.imshow(x_test_denoised[i].squeeze(), cmap='gray')
    plt.axis('off')
    if i == 0: plt.ylabel("Denoised")
    # Original
    plt.subplot(3, n, 2*n+i+1)
    plt.imshow(x_test[i].squeeze(), cmap='gray')
    plt.axis('off')
    if i == 0: plt.ylabel("Original")
plt.suptitle("Denoising Autoencoder: entrada ruidosa → salida limpia")
plt.show()


## 9) Guardar y cargar modelos (ruta local)

In [ ]:

ae_path = "mnist_autoencoder.keras"
denoiser_path = "mnist_denoiser.keras"

autoencoder.save(ae_path)
denoiser.save(denoiser_path)

print("Modelos guardados en:", ae_path, "y", denoiser_path)

autoencoder_loaded = keras.models.load_model(ae_path)
denoiser_loaded = keras.models.load_model(denoiser_path)

# Prueba rápida de carga
loss_loaded = autoencoder_loaded.evaluate(x_test, autoencoder_loaded.predict(x_test, verbose=0), verbose=0)
print("Autoencoder cargado, pérdida (auto-reconstrucción evaluada manualmente):", float(loss_loaded))
